In [33]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:

import pandas as pd
import os
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns
%matplotlib inline
from collections import Counter 
import matplotlib.pyplot as plt

In [35]:
m = pd.read_csv("/content/gdrive/My Drive/finalmerge.csv",index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [36]:
X=m.drop([
               "ArrDelayMinutes",
                "ArrTime",
                "ArrDel15",
                 "Unnamed: 0_x",
                 "Unnamed: 0_y",
                 "Unnamed: 0.1",
                "Origin",
                "Dest",
                "airport_x",
                "airport_y",
                "FlightDate",
                "date_y",
                "date_x",
                "time_x",
                "time_y",
                "windspeedKmph_y",
                "winddirDegree_y",
                "weatherCode_y",
                "precipMM_y",
                "visibility_y",
                "pressure_y",
                "cloudcover_y",
                "DewPointF_y",
                "WindGustKmph_y",
                "tempF_y",
               "WindChillF_y",
               "humidity_y",],axis=1)
X

,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,CRSArrTime,windspeedKmph_x,winddirDegree_x,weatherCode_x,precipMM_x,visibility_x,pressure_x,cloudcover_x,DewPointF_x,WindGustKmph_x,tempF_x,WindChillF_x,humidity_x
0,1,2016,1,1,741.0,0.0,700,0.0,14747,12478,1600,22,255,113,0.0,10,1015,0,32,26,41,32,74
1,1,2016,1,1,705.0,0.0,700,0.0,14747,12892,900,12,31,113,0.0,10,1020,2,30,15,51,47,46
2,1,2016,1,1,658.0,0.0,700,3.0,14747,12892,900,12,31,113,0.0,10,1020,2,30,15,51,47,46
3,1,2016,1,1,723.0,0.0,700,6.0,14747,12478,1500,21,254,113,0.0,10,1016,0,33,24,42,33,74
4,1,2016,1,1,726.0,0.0,700,0.0,14747,12892,1000,10,39,113,0.0,10,1020,9,29,13,53,49,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1815400,3,2017,9,18,2240.0,0.0,2200,0.0,12478,12892,200,5,220,113,0.0,10,1012,1,70,6,74,74,87
1815401,3,2017,9,21,2358.0,1.0,2200,73.0,12478,12892,200,11,179,143,0.0,10,1010,56,63,12,65,65,91
1815402,3,2017,9,22,2240.0,0.0,2200,0.0,12478,12892,200,10,135,113,0.0,10,1010,3,59,12,67,67,75
1815403,3,2017,9,23,2146.0,0.0,2100,3.0,12478,12892,100,6,202,113,0.0,10,1009,4,61,8,68,68,80


In [0]:
Y=m["ArrDel15"]
from sklearn.model_selection import train_test_split  
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = None) 

In [0]:
#Y_train.value_counts()

In [0]:
#resampling to make the 0 and 1 values equal 
X = pd.concat([X_train, Y_train], axis=1)

In [0]:
not_delayed = X[X.ArrDel15==0]
delayed = X[X.ArrDel15==1]

In [0]:
from sklearn.utils import resample
delayed_upsampled = resample(delayed,replace=True, n_samples=len(not_delayed),random_state=27)

In [0]:
upsampled = pd.concat([not_delayed, delayed_upsampled])

In [43]:
upsampled.ArrDel15.value_counts()

1.0    1076937
0.0    1076937
Name: ArrDel15, dtype: int64

In [0]:
#values are equal now
X = upsampled.drop(['ArrDel15'], axis=1)
y = upsampled.ArrDel15

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

In [46]:
X_train

,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,CRSArrTime,windspeedKmph_x,winddirDegree_x,weatherCode_x,precipMM_x,visibility_x,pressure_x,cloudcover_x,DewPointF_x,WindGustKmph_x,tempF_x,WindChillF_x,humidity_x
132262,4,2016,10,22,613.0,0.0,600,13.0,12266,13930,800,9,257,113,0.0,6,1020,1,36,12,43,41,81
504288,2,2016,4,14,2259.0,1.0,2100,81.0,13303,14771,100,13,213,113,0.1,10,1020,61,50,17,53,50,91
942565,1,2017,1,18,1819.0,0.0,1800,0.0,13930,11057,2100,9,337,116,0.0,10,1018,14,42,14,53,52,59
185001,4,2016,11,15,1306.0,0.0,1300,1.0,12892,13930,1900,5,234,113,0.0,10,1012,40,46,8,54,53,77
1481716,2,2017,5,31,920.0,0.0,900,0.0,13204,11618,1200,7,300,122,0.1,10,1017,97,58,9,65,67,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1772004,3,2017,9,8,623.0,0.0,600,0.0,12892,12266,1200,13,74,113,0.0,10,1020,0,54,15,77,77,45
688781,3,2016,7,18,1437.0,1.0,1300,90.0,11292,12478,1800,25,186,200,2.3,8,1014,98,71,40,85,85,76
1582447,2,2017,6,30,1353.0,0.0,1400,3.0,10397,11298,1500,24,175,113,0.0,10,1011,34,74,28,89,90,61
682114,3,2016,7,21,1210.0,1.0,1000,95.0,11292,10397,1500,6,172,200,0.0,9,1018,77,70,8,92,92,51


In [0]:

y_test.value_counts()

0.0    269228
1.0    269156
Name: ArrDel15, dtype: int64

In [0]:
y_train.value_counts()

1.0    807612
0.0    807540
Name: ArrDel15, dtype: int64

In [0]:
#0=1
#decision tree
from sklearn.tree import DecisionTreeClassifier

clf=DecisionTreeClassifier()
clf.fit(X_train,y_train)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [0]:
y_pred2 = clf.predict(X_test)

In [0]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

         0.0       0.98      0.90      0.94    269228
         1.0       0.91      0.98      0.95    269156

    accuracy                           0.94    538384
   macro avg       0.95      0.94      0.94    538384
weighted avg       0.95      0.94      0.94    538384



In [0]:
#extra tree 
from sklearn.ensemble import ExtraTreesClassifier
etc=ExtraTreesClassifier(n_jobs=-1,random_state =None)
etc.fit(X_train,y_train)

In [0]:
Y_predetc=etc.predict(X_test)

In [0]:
target_names=["Class 0","Class 1"]
#from sklearn.metrics import classification_report

print(classification_report(y_test,Y_predetc,target_names=target_names))

In [0]:
Y_predetc
xtdf=pd.DataFrame(Y_predetc)
xtdf[0]
X_test["ArrDelayMinutes"]=m["ArrDelayMinutes"]
X_test

In [0]:
import collections

classified_df = pd.DataFrame(X_test)

classified_df['Arrdel15']=Y_predetc

In [0]:
classified_df
classified_df.to_csv("mod3manipulation.csv")

In [0]:
from google.colab import files
files.download("mod3manipulation.csv")

In [0]:
#xgboost
import xgboost 
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

In [0]:
#not able to run xgboost , my pc crashing
#taking the csv from extra trees as i got the highest accuracy in that
